In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.utils.data
from torch.utils.data import random_split
import torchvision.transforms as transforms
import pytorch_lightning as pl
import pytorch_lightning
from rich import print as rprint

In [2]:
rprint(f'Num of GPU: {torch.cuda.device_count()}')
rprint(f'First Device name: {torch.cuda.get_device_name(0)}')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
BATCH_SIZE = 32
NUM_WORKERS = 4
EPOCHS = 100

In [7]:
class CIFAR100DataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = './data', batch_size: int = 32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
                (0.4914, 0.48216, 0.44653),
                (0.24703, 0.24349, 0.26159)
        )])

    def prepare_data(self):
        torchvision.datasets.CIFAR100(root=self.data_dir, train=True, download=True)
        torchvision.datasets.CIFAR100(root=self.data_dir, train=False, download=True)
        
    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            cifar_full = torchvision.datasets.CIFAR100(root=self.data_dir, train=True, transform=self.transform)
            self.cifar100_train, self.cifar100_val = random_split(cifar_full, [45000, 5000])
            self.dims = tuple(self.cifar100_train[0][0].shape)
        
        if stage == 'test' or stage is None:
            self.cifar100_test = torchvision.datasets.CIFAR100(root=self.data_dir, train=False, transform=self.transform)
            self.dims = tuple(self.cifar100_test[0][0].shape)
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.cifar100_train, batch_size=self.batch_size, shuffle=True)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.cifar100_val, batch_size=self.batch_size, shuffle=False)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.cifar100_test, batch_size=self.batch_size, shuffle=False)

In [8]:
class LitClassifier(pl.LightningModule):
    def __init__(self, num_classes=100):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(128 * 4 * 4, num_classes)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 128 * 4 * 4)
        return self.fc(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch # data batch
        y_hat = self(x) # this is forward pass same as model(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        result = pl.TrainResult(loss)
        result.log('train_loss', loss, on_epoch=True)
        return result
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        result = pl.EvalResult(checkpoint_on=loss)
        result.log('val_loss', loss)
        return result
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=3e-4)



In [9]:
dm = CIFAR100DataModule()
model = LitClassifier()
trainer = pl.Trainer(max_epochs=EPOCHS, gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [10]:
trainer.fit(model, dm)

Files already downloaded and verified
Files already downloaded and verified

  | Name  | Type      | Params
------------------------------------
0 | conv1 | Conv2d    | 896   
1 | conv2 | Conv2d    | 18 K  
2 | conv3 | Conv2d    | 73 K  
3 | pool  | MaxPool2d | 0     
4 | fc    | Linear    | 204 K 


Saving latest checkpoint..



1

In [15]:
trainer.test(model, datamodule=dm)

1